<a href="https://colab.research.google.com/github/InduwaraGayashan001/Generative-AI/blob/main/Chatbot_LLama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [2]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate transformers
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf
!pip -q install sentence_transformers

In [4]:
!pip -q install unstructured
!pip -q install tokenizers
!pip -q install faiss-cpu
!pip -q install langchain-community
!pip -q install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.2 MB/s eta 0:00:00


In [7]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.3/524.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 22.0 MB/s eta 0:00:00


In [9]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Pinecone
import pinecone
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import textwrap
import sys
import os
import torch

In [10]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# Extract Data

In [11]:
URLs =[
    'https://medium.com/@srikar.appal/paper-review-2-multi-digit-number-recognition-from-street-view-imagery-using-deep-convolutional-c915f2bdde67',
    'https://lmsys.org/blog/2023-03-30-vicuna/',
    'https://stability.ai/research/facecraft4d-animated-3d-facial-avatar-generation-from-a-single-image'

]

In [12]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [13]:
len(data)

3

# Split the data into Chunks

In [14]:
text_splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size=1000,
    chunk_overlap=200
)

In [15]:
text_chunks = text_splitter.split_documents(data)

In [16]:
len(text_chunks)

25

In [17]:
text_chunks[0]

Document(metadata={'source': 'https://medium.com/@srikar.appal/paper-review-2-multi-digit-number-recognition-from-street-view-imagery-using-deep-convolutional-c915f2bdde67'}, page_content='Sitemap\nOpen in app\nSign in\nWrite\nSign in\nPaper Review 2 — Multi-digit Number Recognition from Street View Imagery using Deep Convolutional Neural Networks\nSrikar Appalaraju\nSrikar Appalaraju\n3 min read\nApr 13, 2018\n--\nThis paper published in ICLR 2014 is again a rather old paper but following up on the last reviewed paper this paper follows up on it. The authors show that using a CNN they were able to automate street number extraction from street images.')

# Embeddings

In [ ]:
embeddings = HuggingFaceEmbeddings()

# Pinecone Client

In [20]:
!pip install --upgrade pinecone-client

In [28]:
import os
from pinecone import Pinecone, ServerlessSpec

from google.colab import userdata
# Instantiate the Pinecone client
pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))

# Connect to the index
index = pc.Index("llama")

In [29]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

# Upload the embeddings to vectorstore

In [25]:

!pip install --upgrade langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.2 MB/s eta 0:00:00


In [30]:
from langchain_pinecone import PineconeVectorStore

vectorstore = PineconeVectorStore(index,embeddings)

In [31]:
vectorstore.add_texts([t.page_content for t in text_chunks])

['b1bc845b-2a89-4037-8c3a-af2048a0d2e4',
 'd40bb6b7-bcba-4037-a562-e316140af138',
 '426bd902-378d-4e39-9719-d2421eb6c88c',
 '4dcca05a-f436-4310-bc16-5bd34c8a678b',
 '44c708d4-8ce3-4d00-bfbc-96b052ce6d6d',
 'be94bfde-8e8a-4242-bb71-9264d6bb7ad1',
 'ae5db2c5-3225-45be-b4cb-e992180d328e',
 'bce0e9cf-e8f9-4f97-8db3-ba0d1f28eb68',
 '428839ae-a59b-48ac-ac3a-73e1996af541',
 '2acb002c-b253-413f-bebc-84711a7c2d0b',
 '33c568ad-ccc7-48d4-85e6-8f70c66e9bf1',
 '96b5d796-1928-4960-9ad5-ef48cd4bcea3',
 '7de6f540-66e8-4888-850c-9c412593b2e6',
 'add8286b-5942-44a9-858b-863fdbebba19',
 '76f4cb7d-4d6c-4192-be7e-7ba8e56f83a8',
 '0292087b-fe72-4886-b7ac-e76180950c48',
 '9a5a1baa-b882-48ca-a960-90b100089e2f',
 'a43f1442-efa8-4141-9c41-e66ecab6fe22',
 '16702673-45a6-47b0-81af-8bd07968f24d',
 '5b9186d2-eb6a-44b3-9ea6-c616622c939d',
 'a11b7c82-9d38-4aab-9705-6b8d0c2205b6',
 'ce3e9477-f007-490b-8cdf-ce711a2b862f',
 '2841fff6-7f5e-4ef7-9bce-7ffd2f6a1355',
 'f82ea984-369a-46b1-a114-803b3300646e',
 'd5f0c6cb-5a75-

# Create LLM Wrapper

In [ ]:
notebook_login()

In [49]:
model = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model,
    use_auth_token=True)

model = AutoModelForCausalLM.from_pretrained(
    model,
    device_map="auto",
    use_auth_token=True,
    torch_dtype=torch.float16,
    load_in_8bit=True
)

In [ ]:
pipe = pipeline(
    task="text-generation",
    model="meta-llama/Llama-2-7b-hf",
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_new_tokens=512,
    do_sample=True,
    top_k=30,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

In [38]:
llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

/tmp/ipython-input-38-899405379.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})


# Initialize the Retrieval QA

In [39]:
from langchain.chains import RetrievalQA

In [40]:
query = "How good is Vicuna?"
docs = vectorstore.similarity_search(query, k=3)
docs

[Document(id='ae5db2c5-3225-45be-b4cb-e992180d328e', metadata={}, page_content='LMSYS ORG\nProjectsBlogAboutDonationsChatbot Arena (graduated)\nVicuna: An Open-Source Chatbot Impressing GPT-4 with 90%* ChatGPT Quality\nby: The Vicuna Team, Mar 30, 2023\nWe introduce Vicuna-13B, an open-source chatbot trained by fine-tuning LLaMA on user-shared conversations collected from ShareGPT. Preliminary evaluation using GPT-4 as a judge shows Vicuna-13B achieves more than 90%* quality of OpenAI ChatGPT and Google Bard while outperforming other models like LLaMA and Stanford Alpaca in more than 90%* of cases. The cost of training Vicuna-13B is around $300. The code and weights, along with an online demo, are publicly available for non-commercial use.\nVicuna (generated by stable diffusion 2.1)\nAccording to a fun and non-scientific evaluation with GPT-4. Further rigorous evaluation is needed.\nHow Good is Vicuna?'),
 Document(id='bce0e9cf-e8f9-4f97-8db3-ba0d1f28eb68', metadata={}, page_content='V

In [41]:
chain = RetrievalQAWithSourcesChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever()
)

In [42]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [44]:
qa.invoke({"query": query})

{'query': 'How good is Vicuna?',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nLMSYS ORG\nProjectsBlogAboutDonationsChatbot Arena (graduated)\nVicuna: An Open-Source Chatbot Impressing GPT-4 with 90%* ChatGPT Quality\nby: The Vicuna Team, Mar 30, 2023\nWe introduce Vicuna-13B, an open-source chatbot trained by fine-tuning LLaMA on user-shared conversations collected from ShareGPT. Preliminary evaluation using GPT-4 as a judge shows Vicuna-13B achieves more than 90%* quality of OpenAI ChatGPT and Google Bard while outperforming other models like LLaMA and Stanford Alpaca in more than 90%* of cases. The cost of training Vicuna-13B is around $300. The code and weights, along with an online demo, are publicly available for non-commercial use.\nVicuna (generated by stable diffusion 2.1)\nAccording to a fun and non-scientific evaluation with GPT-4. Further rigor

# Chatbot

In [46]:
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print("Exiting")
    sys.exit()
  if user_input == '':
    continue
  result = qa.invoke({"query": user_input})
  print(f"Answer: {result['result']}")

Input Prompt: What is Vicuna?
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Limitations
We have noticed that, similar to other large language models, Vicuna has certain limitations. For instance, it is not good at tasks involving reasoning or mathematics, and it may have limitations in accurately identifying itself or ensuring the factual accuracy of its outputs. Additionally, it has not been sufficiently optimized to guarantee safety or mitigate potential toxicity or bias. To address the safety concerns, we use the OpenAI moderation API to filter out inappropriate user inputs in our online demo. Nonetheless, we anticipate that Vicuna can serve as an open starting point for future research to tackle these limitations.
Release
In our first release, we will share the training, serving, and evaluation code on a GitHub repo: https://github.com/lm-sys/FastChat. We al

SystemExit: 

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
